# Objetivo:

- Recuperar os dados de Empresas e Sócios do site da Receita Federal de forma dinâmica e com tratativa de erro;
- Selecionar Dinâmicamente o diretório mais recente;
- Escolher qual o arquivo que quer baixar ou baixar todos os arquivos;
- Salvar o .zip na data/stage;
- Extrair o arquivo de dentro do .zip na data/stage.

### Import das Bibliotecas

In [ ]:
import requests
import os
import zipfile
from bs4 import BeautifulSoup
from requests.exceptions import RequestException

### Definição do Diretório Bronze

In [ ]:
# Diretório local onde salvar os arquivos
RAW_DIR = "../data/raw"
os.makedirs(RAW_DIR, exist_ok=True)

### Site da Receita Federal

#### Acessar o Site

In [ ]:
# URL base
BASE_URL = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/"

def acessar_url(url, max_tentativas=3):
    for tentativa in range(1, max_tentativas + 1):
        try:
            print(f"Tentativa {tentativa} de {max_tentativas}...")
            resposta = requests.get(url, timeout=10)
            resposta.raise_for_status()
            return resposta.text  # retorna apenas o conteúdo

        except RequestException as e:
            print(f"Erro ao acessar a URL: {e}")
            if tentativa == max_tentativas:
                raise RuntimeError(f"Falha após {max_tentativas} tentativas ao acessar {url}")

In [ ]:
try:
    html = acessar_url(BASE_URL)
    soup = BeautifulSoup(html, "html.parser")
    print("Requisição bem-sucedida!")
except RuntimeError as erro:
    print(erro)

#### Navegar no Site

In [ ]:
# Pegar todos os diretórios
dirs = [a["href"] for a in soup.find_all("a", href=True) if a["href"].endswith("/") and a["href"][0].isdigit()]

In [ ]:
# Ordenar diretórios por nome (o mais recente é o último)
dirs.sort()
latest_dir = dirs[-1]
print(f"Diretório mais recente encontrado: {latest_dir}")

In [ ]:
# Acessar a pasta mais recente
resp_latest = requests.get(BASE_URL + latest_dir)
resp_latest.raise_for_status()
soup_latest = BeautifulSoup(resp_latest.text, "html.parser")

In [ ]:
# Procurar arquivos Empresas.zip e Socios.zip
links = [a["href"] for a in soup_latest.find_all("a", href=True)]

# Recupera todos os arquivos de Empresas e Socios
empresas_files = [l for l in links if "Empresas" in l and l.endswith(".zip")]
socios_files = [l for l in links if "Socios" in l and l.endswith(".zip")]

# Pegar o primeiro arquivo (ou o último, dependendo da sua preferência)
empresas_file = empresas_files[0] # ou empresas_files[-1] para o último
socios_file = socios_files[0]  # ou socios_files[-1]

print(f"Arquivos Empresas encontrados: {empresas_files}")
print(f"Arquivo Empresas selecionado: {empresas_file}")
print(f"Arquivos Socios encontrados: {socios_files}")
print(f"Arquivo Socios selecionado: {socios_file}")

#### Baixar e Extrair Arquivos

In [ ]:
# Função para baixar arquivo
def download_file(file_url, folder):
    filename = file_url.split("/")[-1]
    filepath = os.path.join(folder, filename)
    print(f"Baixando {filename}...")
    r = requests.get(BASE_URL + latest_dir + file_url, stream=True)
    r.raise_for_status()
    with open(filepath, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Salvo em {filepath}")
    return filepath

In [ ]:
# Função para extrair .zip
def extract_zip(zip_path, extract_to):
    print(f"Extraindo {zip_path}...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Arquivos extraídos em {extract_to}")

In [ ]:
# Baixar e descompactar os arquivos
for file in [empresas_file, socios_file]: # Caso queira baixar todos os arquivos trocar por empresas_files e socios_files
    zip_path = download_file(file, RAW_DIR)
    extract_zip(zip_path, RAW_DIR)